In [55]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Load the image
img_path = '_happy_jumping_on_beach-40815.jpg'
img = load_img(img_path)

# Convert the image to a NumPy array
img_array = img_to_array(img)

# Check the shape of the original image array
print("Original Image Shape:", img_array.shape)

# Rescale the image array to fit the model's input shape
img_array = tf.image.resize(img_array, (224, 224))  # Resize the image to (224, 224)
img_array = tf.reshape(img_array, (1, 224, 224, 3))

# Check the shape after resizing
print("Reshaped Image Shape:", img_array.shape)

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define your CNN model
model = models.Sequential()
# ... (rest of your model definition)

# Build the model
model.build(input_shape=(1, 224, 224, 3))

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Display the model architecture
model.summary()

# Load your data and preprocess it (adjust as needed based on your data)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train.csv', target_size=(224, 224), batch_size=32, class_mode='binary')
validation_generator = test_datagen.flow_from_directory('test.csv', target_size=(224, 224), batch_size=32, class_mode='binary')

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Original Image Shape: (213, 320, 3)
Reshaped Image Shape: (1, 224, 224, 3)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


NotADirectoryError: [Errno 20] Not a directory: 'train.csv'

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # Add this import for F.relu
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):  # Fix the method name
        super(SimpleCNN, self).__init__()  # Fix the super() call
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# ... rest of your code ...

# Hyperparameters
num_classes = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.2%}')

Epoch [1/5], Step [100/938], Loss: 0.1213
Epoch [1/5], Step [200/938], Loss: 0.4268
Epoch [1/5], Step [300/938], Loss: 0.0464
Epoch [1/5], Step [400/938], Loss: 0.0628
Epoch [1/5], Step [500/938], Loss: 0.1444
Epoch [1/5], Step [600/938], Loss: 0.0309
Epoch [1/5], Step [700/938], Loss: 0.0550
Epoch [1/5], Step [800/938], Loss: 0.0466
Epoch [1/5], Step [900/938], Loss: 0.0886
Epoch [2/5], Step [100/938], Loss: 0.0260
Epoch [2/5], Step [200/938], Loss: 0.0370
Epoch [2/5], Step [300/938], Loss: 0.0264
Epoch [2/5], Step [400/938], Loss: 0.0091
Epoch [2/5], Step [500/938], Loss: 0.0303
Epoch [2/5], Step [600/938], Loss: 0.0157
Epoch [2/5], Step [700/938], Loss: 0.0493
Epoch [2/5], Step [800/938], Loss: 0.0671
Epoch [2/5], Step [900/938], Loss: 0.0034
Epoch [3/5], Step [100/938], Loss: 0.0240
Epoch [3/5], Step [200/938], Loss: 0.0285
Epoch [3/5], Step [300/938], Loss: 0.0451
Epoch [3/5], Step [400/938], Loss: 0.0293
Epoch [3/5], Step [500/938], Loss: 0.0093
Epoch [3/5], Step [600/938], Loss: